In [3]:
## 학습 데이터에는 존재하지 않거나 희귀하지만 시험 데이터에만 존재하는 범주를 학습하기 위해 학습 데이터와 시험 데이터를 합치기

import pandas as pd
from sklearn import preprocessing

# 학습 데이터를 불러온다.
train = pd.read_csv('cat_train.csv')

# 평가 데이터를 불러온다.
test = pd.read_csv('cat_test.csv')

# 테스트 데이터에는 타겟 열이 없으므로 가짜 타겟 열을 생성한다.
test.loc[:, 'target'] = -1

# 학습 데이터와 평가 데이터를 결합한다.
data = pd.concat([train, test]).reset_index(drop = True)

# 인코딩 할 피쳐의 목록을 만든다.
# id와 target은 포함해서는 안된다.
features = [x for x in train.columns if x not in ['id', 'target']]

# 모든 피쳐에 대해 계산한다.
for feat in features:
    # 각 피쳐에 대해 새로운 LabelEncoder 객체를 생성한다.
    lbl_enc = preprocessing.LabelEncoder()
    
    # 범주형 변수들이므로, 문자열로 fillna()를 호출한 후
    # 모든 정수이든지 실수이든지 모두 문자열로 변환된다.
    temp_col = data[feat].fillna('NONE').astype(str).values
    
    # 학습 데이터와 평가 데이터를 합쳤으므로 fit_transform을 사용하여 한 번에 변환한다.
    data.loc[:, feat] = lbl_enc.fit_transform(temp_col)
    
    
# 합쳤던 학습 데이터와 시험 데이터를 다시 나눈다.
train = data[data.target != -1].reset_index(drop = True)
test = data[data.target == -1].reset_index(drop = True)

In [7]:
## 평가 데이터에 새로 추가되는 범주를 다루기

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')

## 데이터 불러오기
df = pd.read_csv('cat_train.csv')

In [8]:
df.ord_4.fillna('NONE').value_counts()

N       39978
P       37890
Y       36657
A       36633
R       33045
U       32897
M       32504
X       32347
C       32112
H       31189
Q       30145
T       29723
O       25610
B       25212
E       21871
K       21676
I       19805
NONE    17930
D       17284
F       16721
W        8268
Z        5790
S        4595
G        3404
V        3107
J        1950
L        1657
Name: ord_4, dtype: int64

In [10]:
## 2000번 미만으로 나타나는 범주를 희귀 범주로 구분
df.ord_4 = df.ord_4.fillna('NONE')

df.loc[
    df['ord_4'].value_counts()[df['ord_4']].values < 2000,
    'ord_4'] = 'RARE'

df.ord_4.value_counts()

N       39978
P       37890
Y       36657
A       36633
R       33045
U       32897
M       32504
X       32347
C       32112
H       31189
Q       30145
T       29723
O       25610
B       25212
E       21871
K       21676
I       19805
NONE    17930
D       17284
F       16721
W        8268
Z        5790
S        4595
RARE     3607
G        3404
V        3107
Name: ord_4, dtype: int64

In [ ]:
## create_folds.py

# pandas 라이브러리와 사이킷런의 model_selection 모듈을 불러온다.
import pandas as pd
from sklearn import model_selection

if __name__ == '__main__':
    
    # 학습 데이터를 불러온다.
    df = pd.read_csv('cat_train.csv')
    
    # kfold 라는 새 열을 만들고 -1로 채운다.
    df['kfold'] = -1
    
    # 데이터의 행들을 랜덤하게 섞는다.
    df = df.sample(frac = 1).reset_index(drop = True)
    
    # 타겟 변수를 추출한다.
    y = df.target.values
    
    # model_selection 모듈의 kfold 클래스를 초기화 한다.
    kf = model_selection.StratifiedKFold(n_splits = 5)
    
    # kfold 열을 할당한다.
    for f, (t_,v_) in enumerate(kf.split(X=df, y=y)):
        df.loc[v_, 'kfold'] = f
        
    # kfold 열을 가진 새로운 데이터를 저장한다.
    df.to_csv('input/cat_train_fold.csv', index = False)

In [ ]:
import pandas as pd
df = pd.read_csv('input/cat_train_folds.csv')

df.kfold.value_counts()

In [ ]:
print(df[df.kfold == 0].target.value_counts())
print()
print(df[df.kfold == 1].target.value_counts())
print()
print(df[df.kfold == 2].target.value_counts())
print()
print(df[df.kfold == 3].target.value_counts())
print()
print(df[df.kfold == 4].target.value_counts())

In [ ]:
# ohe_logres.py
import pandas as pd

from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    # 폴드 정보가 있는 학습 데이터를 불러온다.\
    df = pd.read_csv('input/cat_train_folds.csv')
    
    # id, target, kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
        f for f in df.columns if f not in ('id', 'target', 'kfold')
    ]
    
    # 모든 NaN 값을 NONE으로 채운다.
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna('NONE')
        
    # 폴드를 이용해 학습 데이터를 나눈다.
    df_train = df[df.kfold != fold].reset_index(drop = True)
        
    # 폴드를 이용해 검증 데이터를 나눈다. 
    df_valid = df[df.kfold != fold].reset_index(drop = True)
        
    # 사이킷런의 OneHotEncoder 객체를 초기화한다.
    ohe = preprocessing.OneHotEncoder()
        
    # ohe를 학습 데이터 + 검증 데이터로 학습한다.
    full_data = pd.concat(
    [df_train[features], df_valid[features]],
    axis = 0
    )
        
    ohe.fit(full_data[features])
        
    # 학습 데이터를 변환한다.
    x_train = ohe.transform(df_train[features])
        
    # 검증 데이터를 변환한다.
    x_valid = ohe.transform(df_valid[features])
        
    # 로지스틱 회귀 모델을 초기화 한다.
    model = linear_model.LogisticRegression()
    # 인코딩 된 학습 데이터로 학습한다.
    model.fit(x_train, df_train.target.values)
        
    # 검증 데이터로 예측한다.
    # AUC를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1의 확률을 사용한다.
    valid_preds = model.predict_proba(x_valid)[:, 1]
        
    # AUC 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
        
    # auc 값을 출력한다.
    print(auc)
    
if __name__ == '__main__':
    # run fold 0
    run(0)

In [ ]:
# ohe_logres.py -> 코드 수정 1
import pandas as pd

from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    # 폴드 정보가 있는 학습 데이터를 불러온다.\
    df = pd.read_csv('input/cat_train_folds.csv')
    
    # id, target, kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
        f for f in df.columns if f not in ('id', 'target', 'kfold')
    ]
    
    # 모든 NaN 값을 NONE으로 채운다.
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna('NONE')
        
    # 폴드를 이용해 학습 데이터를 나눈다.
    df_train = df[df.kfold != fold].reset_index(drop = True)
        
    # 폴드를 이용해 검증 데이터를 나눈다. 
    df_valid = df[df.kfold != fold].reset_index(drop = True)
        
    # 사이킷런의 OneHotEncoder 객체를 초기화한다.
    ohe = preprocessing.OneHotEncoder()
        
    # ohe를 학습 데이터 + 검증 데이터로 학습한다.
    full_data = pd.concat(
    [df_train[features], df_valid[features]],
    axis = 0
    )
        
    ohe.fit(full_data[features])
        
    # 학습 데이터를 변환한다.
    x_train = ohe.transform(df_train[features])
        
    # 검증 데이터를 변환한다.
    x_valid = ohe.transform(df_valid[features])
        
    # 로지스틱 회귀 모델을 초기화 한다.
    model = linear_model.LogisticRegression()
    # 인코딩 된 학습 데이터로 학습한다.
    model.fit(x_train, df_train.target.values)
        
    # 검증 데이터로 예측한다.
    # AUC를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1의 확률을 사용한다.
    valid_preds = model.predict_proba(x_valid)[:, 1]
        
    # AUC 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
        
    # auc 값을 출력한다.
    print(auc)
    
if __name__ == '__main__':
    for fold_ in range(5):
        run(fold_)

In [ ]:
# lbl_rf.py
import pandas as pd

from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing

def run(fold):
    # 폴드 정보가 있는 학습 데이터를 불러온다.\
    df = pd.read_csv('input/cat_train_folds.csv')
    
    # id, target, kfold 열을 제외한 모든 열을 피쳐로 사용한다.
    features = [
        f for f in df.columns if f not in ('id', 'target', 'kfold')
    ]
    
    # 모든 NaN 값을 NONE으로 채운다.
    # 모든 열을 문자열로 변환함을 주목. 범주형으로 처리할 것이므로 원래 타입이 무엇이든지 상관 없다.
    for col in features:
        df.loc[:, col] = df[col].astype(str).fillna('NONE')
        
    # 피쳐를 레이블 인코딩 한다.
    for col in features:
        
        # 각 피쳐에 대해 LabelEncoder 객체를 초기화한다.
        lbi = preprocessing.LabelEncoder()
        
        # 모든 데이터로 label encoder를 학습한다.
        lbi.fit(df[col])
        
        # 모든 데이터를 변환한다.
        df.loc[:, col] = lbi.transform(df[col])
        
    # 폴드를 이용해 학습 데이터를 추출한다.
    df_train = df[df.kfold != fold].reset_index(drop = True)
        
    # 폴드를 이용해 검증 데이터를 추출한다.
    df_valid = df[df.kfold != fold].reset_index(drop = True)
    
    # 학습 데이터를 np.array()로 변환한다.
    x_train = df_train[features].values
    
    # 검증 데이터를 np.array()로 변환한다.
    x_valid = df_valid[features].values
        
    # 랜덤포레스트 모델을 초기화 한다.
    model = ensemble.RandomForestClassifier(n_jobs=-1)
    # 인코딩 된 학습 데이터로 학습한다.
    model.fit(x_train, df_train.target.values)
        
    # 검증 데이터로 예측한다.
    # AUC를 계산할 것이므로 확률 값이 필요하다.
    # 범주 1의 확률을 사용한다.
    valid_preds = model.predict_proba(x_valid)[:, 1]
        
    # AUC 값을 계산한다.
    auc = metrics.roc_auc_score(df_valid.target.values, valid_preds)
        
    # auc 값을 출력한다.
    print(auc)
    
if __name__ == '__main__':
    for fold_ in range(5):
        run(fold_)